In [1]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen

import urllib.request
from bs4 import BeautifulSoup
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

credentials = {'CREDENTIALS_HERE'}
gc = gspread.service_account_from_dict(credentials)

In [2]:
# franchise_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1inma86cQKoPFBZX2txtj6oXqL3VYN58eT0frUTdLRQs/edit#gid=0')

In [7]:
### AVEDA ###

url = 'https://aveda.edu/locations/'
req = urllib.request.Request(url)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

all_locations = soup.find_all('div', class_='location-tile')

aveda_dict = {}
aveda_df = pd.DataFrame()

for i in range(len(all_locations)):
    aveda_dict['Website'] = all_locations[i].find(href=True)['href']
    state = all_locations[i].find('div', class_='label').text
    aveda_dict['State'] = state
    city = all_locations[i].find('span', class_='city').text
    aveda_dict['City'] = city
    # aveda_dict['Phone'] = all_locations[i].find('span', class_='phone').text
    aveda_dict['Address'] = all_locations[i].find('span', class_='address').text + ' - ' + city + ', ' + state
    school_name = all_locations[i].find('span', class_='school').text
    if city in school_name:
        aveda_dict['School Name'] = school_name.replace(city, ' - '+ city).replace('  ',' ').replace(' - - ', ' - ').replace('(', '').replace(')', '')
    else:
        aveda_dict['School Name'] = school_name + ' - ' + city

    aveda_df = aveda_df.append(aveda_dict, ignore_index=True)

aveda_df = aveda_df.reindex(['Company ID', 'School Name', 'City', 'State', 'Address', 'Website'], axis=1)

aveda_sheet = franchise_wb.worksheet('Aveda')
set_with_dataframe(aveda_sheet, aveda_df)

In [3]:
### Summit Salon Academy ###

url = 'https://summitsalonacademy.com/academies/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

summit_salon_dict = {}
summit_salon_df = pd.DataFrame()

all_locations = soup.find_all('div', class_='et_pb_blurb_content')

for i in range(len(all_locations)):
    this_location = BeautifulSoup(str(all_locations[i]))

    city = this_location.find(class_='et_pb_module_header').text.split(', ')[0].title()
    summit_salon_dict['City'] = city
    summit_salon_dict['State'] = this_location.find(class_='et_pb_module_header').text.split(', ')[1]
    summit_salon_dict['Programs'] = str(this_location.find_all('li')).replace('<li>', '').replace('</li>', '')
    summit_salon_dict['Website'] = this_location.find(href=True)['href']
    summit_salon_dict['Name'] = 'Summit Salon Academy - ' + city

    summit_salon_df = summit_salon_df.append(summit_salon_dict, ignore_index=True)

summit_salon_df = summit_salon_df.reindex(['Company ID', 'Name', 'City', 'State', 'Programs', 'Website'], axis=1)

salon_summit = franchise_wb.worksheet('Summit Salon')
set_with_dataframe(salon_summit, summit_salon_df)

In [137]:
### Toni & Guy ###

url = 'https://careerschoolnow.org/colleges/toni-guy-hairdressing-academy'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

toni_guy_dict = {}
toni_guy_df = pd.DataFrame()

all_locations = soup.find_all('div', class_='campus-listing')

for i in range(len(all_locations)):
    this_location = all_locations[i]

    toni_guy_dict['Website'] = this_location.find(href=True)['href']
    toni_guy_dict['Name'] = this_location.find(href=True)['title'].replace('-', ' - ')
    toni_guy_dict['Address'] = str(this_location.find('p', class_='full_address')).replace('<p class="full_address">', '').replace('</p>', '').replace('<br/>', ' - ')
    toni_guy_dict['City'] = str(this_location.find('p', class_='full_address')).replace('<p class="full_address">', '').replace('</p>', '').split('<br/>')[1].split(', ')[0]
    state = str(this_location.find('p', class_='full_address')).replace('<p class="full_address">', '').replace('</p>', '').split('<br/>')[1].split(', ')[1]
    toni_guy_dict['State'] = re.sub('\d', '', state)

    toni_guy_df = toni_guy_df.append(toni_guy_dict, ignore_index = True)

toni_guy_df = toni_guy_df.reindex(['Company ID', 'Name', 'Address', 'City', 'State', 'Website'], axis=1)

toni_guy = franchise_wb.worksheet('Toni&Guy')
set_with_dataframe(toni_guy, toni_guy_df)

In [212]:
### SPEC Franchise ###

url = 'https://www.specfranchise.com/about-us/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

spec_dict = {}
spec_df = pd.DataFrame()

all_locations = soup.find_all('div', class_='shcool-info-content')

for i in range(len(all_locations)):
    this_location = all_locations[i]

    name = this_location.find('div', class_='site-notice').text.replace('\n', '').replace('  ', '').strip().replace('TSPA ', 'TSPA - ').replace('ESI ', 'ESI - ')
    spec_dict['Name'] = name
    address = this_location.find('div', class_='address').text.replace('Location: ', '').replace(name +', ', '').replace(', USA', '')
    spec_dict['City'] = name.replace('TSPA - ', '').replace('ESI - ', '')
    spec_dict['Address'] = address
    spec_dict['State'] = re.sub('\d', '', address.split(', ')[-1]).replace(' ', '')
    spec_dict['Website'] = this_location.find(href=True)['href']

    spec_df = spec_df.append(spec_dict, ignore_index = True)

spec_df = spec_df.reindex(['Company ID', 'Name', 'Address', 'City', 'State', 'Website'], axis=1)

spec_sheet = franchise_wb.worksheet('SPEC')
set_with_dataframe(spec_sheet, spec_df)

In [294]:
### TRICOCI ###
url = 'https://www.tricociuniversity.edu/cosmetology-esthetics-barbering-schools/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

tricoci_dict = {}
tricoci_df = pd.DataFrame()

illinois = soup.find_all('ul', id='menu-illinois-side-navigation')
illinois_schools = BeautifulSoup(str(illinois)).find_all('li')

for i in range(len(illinois_schools)):
    this_location = illinois_schools[i]

    tricoci_dict['Name'] = 'Tricoci - ' + this_location.text
    tricoci_dict['Website'] = 'tricociuniversity.edu' + this_location.find(href=True)['href']
    tricoci_dict['City'] = this_location.text
    tricoci_dict['State'] = 'Illinois'

    tricoci_df = tricoci_df.append(tricoci_dict, ignore_index = True)

indiana = soup.find_all('ul', id='menu-indiana-side-navigation')
indiana_schools = BeautifulSoup(str(indiana)).find_all('li')

for i in range(len(indiana_schools)):
    this_location = indiana_schools[i]

    tricoci_dict['Name'] = 'Tricoci - ' + this_location.text
    tricoci_dict['Website'] = 'tricociuniversity.edu' + this_location.find(href=True)['href']
    tricoci_dict['City'] = this_location.text
    tricoci_dict['State'] = 'Indiana'

    tricoci_df = tricoci_df.append(tricoci_dict, ignore_index = True)

wisconsin = soup.find_all('ul', id='menu-wisconsin-side-navigation')
wisconsin_schools = BeautifulSoup(str(wisconsin)).find_all('li')

for i in range(len(wisconsin_schools)):
    this_location = wisconsin_schools[i]

    tricoci_dict['Name'] = 'Tricoci - ' + this_location.text
    tricoci_dict['Website'] = 'tricociuniversity.edu' + this_location.find(href=True)['href']
    tricoci_dict['City'] = this_location.text
    tricoci_dict['State'] = 'Wisconsin'

    tricoci_df = tricoci_df.append(tricoci_dict, ignore_index = True)

tricoci_df = tricoci_df.reindex(['Company ID', 'Name', 'City', 'State', 'Website'], axis=1)

tricoci_sheet = franchise_wb.worksheet('Tricoci')
set_with_dataframe(tricoci_sheet, tricoci_df)

In [ ]:
url = 'https://www.specfranchise.com/about-us/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

In [317]:
import json 
import urllib3 
from base64 import encode 
 
 
http = urllib3.PoolManager() 
 
user = 'EMAIL'
passwd = 'PASSWORD'
hash_string = f"{user}:{passwd}"
hash_string = hash_string.encode(encoding='UTF-8',errors='strict')
headers = { 
"Authorization": f"Basic {hash_string}" 
} 
 
url = 'https://sa1.connectandsell.com/ConnectAndSell/'
# Make your request 
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

# Convert JSON to dict 
# response_data = json.loads(response.data.decode("utf-8")) 
 
# print(response_data) 